In [1]:
# import required libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import pickle

In [2]:
#load Engaged dataframe
eng_df = pd.read_csv('eng_df.csv',index_col='MSISDN/Number')

In [3]:
# Load experience df
experiance_df = pd.read_csv('experiance_df.csv',index_col='MSISDN/Number')

In [4]:
eng_df.shape

(106856, 4)

In [5]:
experiance_df.shape

(106856, 5)

### Task 4 - Satisfaction Analysis

1. engagement score to each user. Consider the engagement score as the Euclidean distance between the user data point & the less engaged cluster (use the first clustering for this) 

2. experience score to each user. Consider the experience score as the Euclidean distance between the user data point & the worst experience’s cluster.

In [6]:
## Engagement Score
lowest_engagement = eng_df.groupby('cluster-engagement').get_group(0).mean()
lowest_engagement

Bearer Id             1.239300e+00
Dur. (ms).1           1.145312e+08
Total                 6.107454e+08
cluster-engagement    0.000000e+00
dtype: float64

In [7]:
def get_engagement_score(df, lowest):
    x = float(lowest['Bearer Id'])
    y = float(lowest['Dur. (ms).1'])
    z = float(lowest['Total'])
    new_df = df.copy()
    new_df['engagement score'] = ((df['Bearer Id'] - x)**2 + (df['Dur. (ms).1'] - y)**2 + (df['Total'] - z)**2)**0.5
    return new_df
engagement_scored_df = get_engagement_score(eng_df, lowest_engagement)
engagement_scored_df.head()

,Bearer Id,Dur. (ms).1,Total,cluster-engagement,engagement score
MSISDN/Number,,,,,
3.360100e+10,1,116720140,8.786906e+08,0,2.679541e+08
3.360100e+10,1,181230963,1.568596e+08,0,4.587604e+08
3.360100e+10,1,134969374,5.959665e+08,0,2.522172e+07
3.360101e+10,1,49878024,4.223207e+08,0,1.992082e+08
3.360101e+10,2,37104453,1.457411e+09,0,8.501985e+08


In [8]:
lowest_experiance = experiance_df.select_dtypes(exclude='object').groupby('cluster-experiance').get_group(0).mean()
lowest_experiance

Total TCP Retrans     5.956913e+06
Total Throughput      3.654432e+04
Total RTT             1.509119e+02
cluster-experiance    0.000000e+00
dtype: float64

In [9]:
def get_experiance_score(df, low):
    x = float(low['Total RTT'])
    y = float(low['Total TCP Retrans'])
    z = float(low['Total Throughput'])
    new_df = df.copy()
    new_df['experience score'] = ((df['Total RTT'] - x)**2 + (df['Total TCP Retrans'] - y)**2 \
                              + (df['Total Throughput'] - z)**2 )**0.5
    return new_df
experiance_scored_df = get_experiance_score(experiance_df, lowest_experiance)
experiance_scored_df.head()

,Handset Type,Total TCP Retrans,Total Throughput,Total RTT,cluster-experiance,experience score
MSISDN/Number,,,,,,
3.360100e+10,Huawei P20 Lite Huawei Nova 3E,2660.0,74.0,46.0,0,5.954364e+06
3.360100e+10,Apple iPhone 7 (A1778),2660.0,96.0,31.0,0,5.954364e+06
3.360100e+10,undefined,2660.0,96.0,33.0,0,5.954364e+06
3.360101e+10,Apple iPhone 5S (A1457),2396.0,408.0,84.0,0,5.954626e+06
3.360101e+10,Apple iPhone Se (A1723),9373492.0,80790.0,119.0,0,3.416866e+06


#### Task 4.2 - Consider the average of both engagement & experience scores as the satisfaction score & report the top 10 satisfied customer

In [10]:
satisfaction_df = pd.merge(engagement_scored_df["engagement score"], experiance_scored_df['experience score'], on='MSISDN/Number')
satisfaction_df['satisfaction score'] = (satisfaction_df['engagement score'] + satisfaction_df['experience score']) / 2

In [11]:
satisfaction_df.sort_values(by='satisfaction score', ascending=False).head(10)

,engagement score,experience score,satisfaction score
MSISDN/Number,,,
3.362632e+10,5.444797e+11,1.046982e+10,2.774748e+11
3.362578e+10,2.005591e+10,5.621638e+06,1.003077e+10
3.361489e+10,1.284104e+10,2.581316e+06,6.421813e+09
3.376054e+10,1.210245e+10,2.858253e+07,6.065515e+09
3.366716e+10,9.977985e+09,3.353525e+06,4.990669e+09
3.366773e+10,1.626035e+09,8.348252e+09,4.987144e+09
3.365908e+10,1.052080e+09,7.935903e+09,4.493992e+09
3.366098e+10,1.590823e+09,7.368183e+09,4.479503e+09
3.367588e+10,8.693661e+09,3.199662e+07,4.362829e+09


#### Task 4.3 - Build a regression model of your choice to predict the satisfaction score of a customer.

In [12]:
regretion_df = pd.merge(eng_df[['Bearer Id', 'Dur. (ms).1', 'Total']],\
                        experiance_df[['Total RTT','Total TCP Retrans', 'Total Throughput']],\
                       on='MSISDN/Number')
regretion_df = pd.merge(regretion_df,satisfaction_df['satisfaction score'], on='MSISDN/Number' )
regretion_df.head()

,Bearer Id,Dur. (ms).1,Total,Total RTT,Total TCP Retrans,Total Throughput,satisfaction score
MSISDN/Number,,,,,,,
3.360100e+10,1,116720140,8.786906e+08,46.0,2660.0,74.0,1.369542e+08
3.360100e+10,1,181230963,1.568596e+08,31.0,2660.0,96.0,2.323574e+08
3.360100e+10,1,134969374,5.959665e+08,33.0,2660.0,96.0,1.558804e+07
3.360101e+10,1,49878024,4.223207e+08,84.0,2396.0,408.0,1.025814e+08
3.360101e+10,2,37104453,1.457411e+09,119.0,9373492.0,80790.0,4.268077e+08


In [26]:
X = regretion_df[['Bearer Id', 'Dur. (ms).1', 'Total','Total RTT','Total TCP Retrans', 'Total Throughput']].values
X = StandardScaler().fit_transform(X)
X.shape

(106853, 6)

In [27]:
y = regretion_df[['satisfaction score']].values
y = StandardScaler().fit_transform(y)
y.shape

(106853, 1)

In [28]:
model = LinearRegression().fit(X, y)
model.score(X, y)

0.9810735481703408

### Dump the model by pickle

In [17]:
pickle.dump(model,open('Telecom_satisfaction_model.sav','wb'))

#### Task 4.4 - Run a k-means (k=2) on the engagement & the experience score .

In [16]:
eng_exp_df = satisfaction_df[['engagement score', 'experience score']]

In [17]:
min_max_scaler = preprocessing.MinMaxScaler()
eng_exp_values = eng_exp_df.values

scalled_values = min_max_scaler.fit_transform(eng_exp_values)
eng_exp_normalized = pd.DataFrame(data=scalled_values, columns=eng_exp_df.columns)


kmeans = KMeans(n_clusters=2).fit(eng_exp_normalized)

#### Task 4.5 - Aggregate the average satisfaction & experience score per cluster.

In [18]:
eng_exp_df['clusters'] = kmeans.labels_
eng_exp_df['clusters'].value_counts()

clusters
0    106624
1       229
Name: count, dtype: int64

In [19]:
result = eng_exp_df.groupby('clusters').mean()
result

,engagement score,experience score
clusters,,
0,3.591119e+08,1.023420e+07
1,3.109622e+09,2.986994e+09


In [20]:
satisfaction_df = satisfaction_df.reset_index()
satisfaction_df.head()
satisfaction_df.to_csv('Telecom_User_Satisfaction.csv')